In [77]:
import geopandas as gpd
import pandas as pd
from pathlib import Path

In [11]:
gdf = gpd.read_file("/mnt/store-lidarhd/projet-LHD/IA/BDForet/Data/PureForestID/metadata/PureForestID-patches.gpkg")
gdf = gdf[gdf.split.isin(["test","val"])]
gdf = gdf[["split", "patch_num", "bdforet_identifier","bdforet_cat_code", "bdforet_cat_name","inventory_species_label", ]]
print(gdf.columns)
gdf.head()

Index(['split', 'patch_num', 'bdforet_identifier', 'bdforet_cat_code',
       'bdforet_cat_name', 'inventory_species_label'],
      dtype='object')


,split,patch_num,bdforet_identifier,bdforet_cat_code,bdforet_cat_name,inventory_species_label
0,val,00035_00000,FORESTIE0000000000100093,FF1-14-14,Forêt fermée de robinier pur,14
1,val,00035_00001,FORESTIE0000000000100093,FF1-14-14,Forêt fermée de robinier pur,14
2,val,00035_00002,FORESTIE0000000000100093,FF1-14-14,Forêt fermée de robinier pur,14
3,val,00035_00003,FORESTIE0000000000100093,FF1-14-14,Forêt fermée de robinier pur,14
4,val,00035_00004,FORESTIE0000000000100093,FF1-14-14,Forêt fermée de robinier pur,14


In [79]:
PREDICTIONS_FOLDER = Path("/mnt/store-lidarhd/projet-LHD/IA/MYRIA3D-SHARED-WORKSPACE/CGaydon/runs/2023-11-03/17-46-48/")
PREDICTIONS_FILE = PREDICTIONS_FOLDER / "predictions.csv"
df = pd.read_csv(PREDICTIONS_FILE)
# df["patch_id"] = df["patch_id"].apply(lambda patch_id: patch_id + "_BDF_IF")
df.head(n=10) # duplicates !
df = df.drop_duplicates("patch_id",keep="first")
# df.head()

In [80]:
merge = gdf.merge(df, left_on="patch_num", right_on="patch_id", how="inner")
# merge["OK"] = (merge["preds"] == merge["targets"]).astype(int)
# merge["NOK"] = (merge["preds"] != merge["targets"]).astype(int)
print(len(merge))
# merge.head()

22428


In [81]:
groups = merge.groupby(["bdforet_identifier","bdforet_cat_code","targets","preds"]).size().reset_index()
pivot = groups.pivot(index=["bdforet_identifier","targets"],columns="preds",values=0).fillna(0).astype(int)
pivot["consensus"] = pivot.idxmax(axis=1)
pivot = pivot.reset_index()
pivot["accurate"] = pivot["consensus"] == pivot["targets"]
pivot.to_csv(PREDICTIONS_FOLDER/"predictions-pivot-by-polygon.csv")
pivot

preds,bdforet_identifier,targets,0,1,2,3,4,6,7,8,9,10,11,consensus,accurate
0,FORESTIE0000000000403437,0,0,0,10,58,0,2,0,30,0,3,0,3,False
1,FORESTIE0000000000410030,8,0,0,0,0,0,0,0,280,0,0,0,8,True
2,FORESTIE0000000000600707,3,6,82,1095,1281,124,386,47,0,0,3,0,3,True
3,FORESTIE0000000000622465,6,12,2,1,13,9,591,440,22,3,34,1,6,True
4,FORESTIE0000000001316805,7,0,0,0,0,0,0,405,0,0,0,0,7,True
5,FORESTIE0000000001319907,7,0,0,0,0,0,0,19,0,0,0,0,7,True
6,FORESTIE0000000001504926,3,0,0,0,36,0,0,0,0,0,0,0,3,True
7,FORESTIE0000000001505158,0,503,34,0,7,0,0,0,0,0,0,0,0,True
8,FORESTIE0000000001919642,11,0,0,0,1,0,0,0,0,6,3,109,11,True
9,FORESTIE0000000002323832,9,0,1,0,1,0,0,0,0,39,0,0,9,True


In [83]:
synthesis = pivot.groupby("targets").agg({"accurate":"mean","bdforet_identifier":"size"}).round(2)
synthesis.to_csv(PREDICTIONS_FOLDER/"predictions-accuracy-by-class.csv")
synthesis


preds,accurate,bdforet_identifier
targets,,
0,0.33,3
1,0.33,3
2,0.92,12
3,0.71,7
4,1.00,3
6,1.00,4
7,1.00,2
8,1.00,1
9,1.00,5
